<table style="border: none" align="left">
    <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/cars-4-you/master/static/images/logo.png" width="200" alt="Icon"></th>
       <th style="border: none"><font face="verdana" size="5" color="black"><b>Use python function to predict customer satisfaction.</b></th>
   </tr>
</table>

<img align=left src="https://github.com/pmservice/cars-4-you/raw/master/static/images/ai_function.png" alt="Icon" width="664">

Keras model and AI function to determine if comment is a complain.

Contents

- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Load and explore data](#load)
- [3. Create Keras model using TensorFlow backend](#model)
- [4. Store the model in the repository](#persistence)
- [5. Deploy the model](#deployment)
- [6. Python function](#ai_function)
- [7. Payload logging for AI function](#ai_function)

<a id="setup"></a>
## 0. Setup

Install TensorFlow version 1.5 and newest version of watson-machine-learning-client.

In [1]:
!pip install --upgrade tensorflow==1.5

    100% |████████████████████████████████| 44.4MB 22kB/s  eta 0:00:01
Requirement not upgraded as not directly required: protobuf>=3.4.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
Requirement not upgraded as not directly required: six>=1.10.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
Requirement not upgraded as not directly required: numpy>=1.12.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
    100% |████████████████████████████████| 92kB 8.6MB/s eta 0:00:01
    100% |████████████████████████████████| 3.0MB 346kB/s eta 0:00:01
Requirement not upgraded as not directly required: wheel>=0.26 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
Requirement not upgraded as not directly required: setuptools in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from protobuf>=3.4.0->tensorflow==1.5)
Requirement not upgraded as not direct

In [2]:
!rm -rf $PIP_BUILD/watson-machine-learning-client
!pip install --upgrade watson-machine-learning-client

    100% |████████████████████████████████| 942kB 1.1MB/s eta 0:00:01
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tqdm in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: urllib3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-

**TIP:** Restart the kernel (Kernel -> Restart)

#### Watason Machine Learning client

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2018-09-26 09:59:04,078 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


**TIP:** Put your Watson Machine Learning credentials here.

In [2]:
wml_credentials = {
  "apikey": "***",
  "instance_id": "***",
  "password": "***",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***"
}


In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
client.version

'1.0.331'

<a id="introduction"></a>
## 1. Introduction

This notebook trains a **Keras** (TensorFlow) model to predict customer satisfaction based on provided feedback. Notebook also shows usage of **python function** for deep learning model data preprocessing required before model scoring.

<a id="load"></a>
## 2. Load and explore data

In [5]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [6]:
import wget

link_to_data = 'https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/cars-4-you/data/car_rental_training_data.csv'
filename = wget.download(link_to_data)

print(filename)

car_rental_training_data.csv


In this section the data is loaded as pandas dataframe.

In [11]:
import pandas as pd

data = pd.read_csv(filename, sep=';') 
data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,83,Female,M,2,48.85,Inactive,Yes,I thought the representative handled the initi...,0,Product: Availability/Variety/Size,Free Upgrade
1,1307,Female,M,0,55.00,Inactive,No,I have had a few recent rentals that have take...,0,Product: Availability/Variety/Size,Voucher
2,1737,Male,M,0,42.35,Inactive,Yes,car cost more because I didn't pay when I rese...,0,Product: Availability/Variety/Size,Free Upgrade
3,3721,Male,M,2,61.71,Inactive,Yes,I didn't get the car I was told would be avail...,0,Product: Availability/Variety/Size,Free Upgrade
4,11,Male,S,2,56.47,Active,No,If there was not a desired vehicle available t...,1,Product: Availability/Variety/Size,NaN


**Note:** 0 - not satisfied, 1 - satisfied

Extract needed columns and count number of records.

In [12]:
complain_data = data[['Customer_Service', 'Satisfaction']]

In [13]:
print(complain_data.count())

Customer_Service    486
Satisfaction        486
dtype: int64


<a id="model"></a>
## 3. Create Keras model using TensorFlow backend


In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import os
import numpy
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

### 3.1 Prepare data

In [15]:
max_fatures = 500

for idx,row in complain_data.iterrows():
    row[0] = row[0].replace('rt',' ')

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(complain_data['Customer_Service'].values)
X = tokenizer.texts_to_sequences(complain_data['Customer_Service'].values)

maxlen = 50

X = pad_sequences(X, maxlen=maxlen)
print(X.shape)

(486, 50)


Split into train and test datasets.

In [16]:
Y = complain_data['Satisfaction'].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(325, 50) (325,)
(161, 50) (161,)


### 3.2 Design and train model

Create the network definition based on Gated Recurrent Unit (Cho et al. 2014).

In [17]:
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(max_fatures, embedding_vector_length, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            16000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 72,405
Trainable params: 72,405
Non-trainable params: 0
_________________________________________________________________
None


Train the model.

In [18]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=64)

Train on 325 samples, validate on 161 samples
Epoch 1/20
325/325 [==============================] - 1s 4ms/step - loss: 0.6899 - acc: 0.5477 - val_loss: 0.6880 - val_acc: 0.5342
Epoch 2/20
325/325 [==============================] - 1s 2ms/step - loss: 0.6792 - acc: 0.5785 - val_loss: 0.6817 - val_acc: 0.5342
Epoch 3/20
325/325 [==============================] - 1s 2ms/step - loss: 0.6615 - acc: 0.5785 - val_loss: 0.6909 - val_acc: 0.5342
Epoch 4/20
325/325 [==============================] - 1s 2ms/step - loss: 0.6541 - acc: 0.5785 - val_loss: 0.6760 - val_acc: 0.5342
Epoch 5/20
325/325 [==============================] - 1s 2ms/step - loss: 0.6270 - acc: 0.6123 - val_loss: 0.6378 - val_acc: 0.7205
Epoch 6/20
325/325 [==============================] - 1s 2ms/step - loss: 0.6035 - acc: 0.7877 - val_loss: 0.6149 - val_acc: 0.7516
Epoch 7/20
325/325 [==============================] - 1s 2ms/step - loss: 0.5718 - acc: 0.8246 - val_loss: 0.5667 - val_acc: 0.7516
Epoch 8/20
325/325 [==========

In [19]:
print("Best accuracy on test: %3.3f" % numpy.array(history.history['val_acc']).max())

Best accuracy on test: 0.957


**Note:** For purpose of this demo model tuning has been skipped.

Store and archive the model on notebook filesystem.

In [20]:
# evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Evaluation Accuracy: %.2f%%" % (scores[1]*100))

Evaluation Accuracy: 95.65%


In [21]:
filename = 'complain_model.h5'
model.save(filename)

#compress keras model
tar_filename = filename + ".tgz"
cmdstring = "tar -zcvf " + tar_filename + " " + filename
os.system(cmdstring);

In [22]:
!ls -lat

total 1768
-rw-r----- 1 dsxuser dsxuser 815014 Sep 26 09:58 complain_model.h5.tgz
drwxr-x--- 2 dsxuser dsxuser   4096 Sep 26 09:58 .
-rw-r----- 1 dsxuser dsxuser 903944 Sep 26 09:58 complain_model.h5
-rw-r----- 1 dsxuser dsxuser  79518 Sep 26 09:53 car_rental_training_data.csv
drwx------ 1 dsxuser dsxuser   4096 Sep 26 09:47 ..


<a id="persistence"></a>
## 4. Store the model in the repository

In [23]:
model_props = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Satisfaction Prediction Model",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5",
    client.repository.ModelMetaNames.RUNTIME_NAME: "python",
    client.repository.ModelMetaNames.RUNTIME_VERSION: "3.5",
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.1.3'}]
}

published_model_details = client.repository.store_model(model=tar_filename, meta_props=model_props)       

In [24]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

c33a8a25-703f-41ac-9b07-f0db8ddc6ed6


<a id="deployment"></a>
## 5. Deploy the model

### 5.1 Create deployment

In [25]:
deployment = client.deployments.create(model_uid, 'CARS4U - Satisfaction Prediction Model Deployment')



#######################################################################################

Synchronous deployment creation for uid: 'c33a8a25-703f-41ac-9b07-f0db8ddc6ed6' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8c8043fc-97ab-4dca-a8dc-d95848ae8690'
------------------------------------------------------------------------------------------------




In [26]:
client.deployments.list()

------------------------------------  -------------------------------------------------  ------  --------------  ------------------------  --------------  -------------
GUID                                  NAME                                               TYPE    STATE           CREATED                   FRAMEWORK       ARTIFACT TYPE
8c8043fc-97ab-4dca-a8dc-d95848ae8690  CARS4U - Satisfaction Prediction Model Deployment  online  DEPLOY_SUCCESS  2018-09-26T09:59:09.549Z  tensorflow-1.5  model
c71edbca-727c-4e7f-9cd7-99929c5b63bf  CARS4U - Action Model Deployment                   online  DEPLOY_SUCCESS  2018-09-20T09:45:22.524Z  mllib-2.1       model
10e2f722-b2b2-4ece-89d4-b691c5ad0d04  drug-selection deployment                          online  UPDATE_SUCCESS  2018-09-20T08:14:17.933Z  mllib-2.1       model
a65b7a1c-3563-4748-a05d-9fbddb2f1c0c  product line deployment                            online  DEPLOY_SUCCESS  2018-09-20T08:12:33.616Z  mllib-2.1       model
------------------

### 5.2 Score the model

Let's see if our deployment works.

In [27]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)

In [28]:
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/e30fe554-6e3e-4e0e-af06-90f93686f358/deployments/8c8043fc-97ab-4dca-a8dc-d95848ae8690/online


In [29]:
index = 5

scoring_data = X[index].tolist()
print(X_test[index])
print(Y_test[index])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   4  43  19   3 242  25
 235   6  22 353   1 237  12  10   8 107  14  34   5  56]
1


In [30]:
scoring_payload = {'values': [scoring_data]}
scores = client.deployments.score(scoring_endpoint, scoring_payload)

Let's print scoring results.

In [33]:
print(str(scores))

{'values': [[[0.00024344005214516073], [0], [0.00024344005214516073]]], 'fields': ['prediction', 'prediction_classes', 'probability']}


<a id="function"></a>
## 6. Python function

Let's define function that does data preprocessing and model scoring for us. As noticed above model expects numerical input, so the text comment needs to be preprocessed.

### 6.1 Definition

Define some generic parameters our function will use to score the model.

#### Parameters

In [34]:
ai_params = {
    'scoring_endpoint': scoring_endpoint,
    'wml_credentials': wml_credentials,
    'word_index': tokenizer.word_index,
}

In [42]:
def score_generator(params=ai_params):

    def score(payload):
        import re
        from watson_machine_learning_client import WatsonMachineLearningAPIClient
        client = WatsonMachineLearningAPIClient(params['wml_credentials'])
        
        max_fatures = 500
        maxlen = 50

        preprocessed_records = []
        complain_data = payload['values']
        word_index = params['word_index']

        for data in complain_data:
            comment = data[0]
            cleanString = re.sub(r"[!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~]", "", comment)
            splitted_comment = cleanString.split()[:maxlen]
            hashed_tokens = []

            for token in splitted_comment:
                index = word_index.get(token, 0)
                if index < 501 and index > 0:
                    hashed_tokens.append(index)

            hashed_tokens_size = len(hashed_tokens)
            padded_tokens = [0]*(maxlen-hashed_tokens_size) + hashed_tokens
            preprocessed_records.append(padded_tokens)

        scoring_payload = {'values': preprocessed_records}
        
        return client.deployments.score(params['scoring_endpoint'], scoring_payload)
        
        
    return score


#### Function

#### Test locally

In [45]:
sample_data = {
    'fields': ['feedback'],
    'values': [
        ['delayed shuttle, almost missed flight, bad customer service'],
        ['The car was great and they were able to provide all features I wanted with limited time they had.']
    ]
}

In [46]:
score = score_generator()
score(sample_data)

{'fields': ['prediction', 'prediction_classes', 'probability'],
 'values': [[[0.014653529971837997], [0], [0.014653529971837997]],
  [[0.9985828399658203], [1], [0.9985828399658203]]]}

**Note:** 0 - not satisfied. 1 - satisfied

### 6.2 Function storing

In [47]:
client.repository.FunctionMetaNames.show()

------------------  ----  --------
META_PROP NAME      TYPE  REQUIRED
NAME                str   Y
DESCRIPTION         str   N
RUNTIME_UID         str   N
INPUT_DATA_SCHEMA   dict  N
OUTPUT_DATA_SCHEMA  dict  N
TAGS                list  N
------------------  ----  --------


In [48]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'CARS4U - Satisfaction Prediction - AI Function',
}

function_details = client.repository.store_function(meta_props=meta_data, function=score_generator)

No RUNTIME_UID passed. Creating default runtime... SUCCESS

Successfully created default runtime with uid: b3e8f953-012a-4869-a3b7-0bb59f9b63dd


In [49]:
client.repository.list_functions()

------------------------------------  ----------------------------------------------  ------------------------  ------
GUID                                  NAME                                            CREATED                   TYPE
6bf467f1-eb7d-4bfc-8162-4a5925e17eb5  CARS4U - Satisfaction Prediction - AI Function  2018-09-26T10:07:04.352Z  python
------------------------------------  ----------------------------------------------  ------------------------  ------


### 6.3 Function deployment

In [50]:
function_uid = client.repository.get_function_uid(function_details)

function_deployment_details = client.deployments.create(artifact_uid=function_uid, name='CARS4U - Satisfaction Prediction - AI Function Deployment')



#######################################################################################

Synchronous deployment creation for uid: '6bf467f1-eb7d-4bfc-8162-4a5925e17eb5' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='cbfa1a73-9448-4c8b-9a88-5b5d45d4905f'
------------------------------------------------------------------------------------------------




### Score function

In [51]:
ai_function_scoring_endpoint = client.deployments.get_scoring_url(function_deployment_details)

print(ai_function_scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/e30fe554-6e3e-4e0e-af06-90f93686f358/deployments/cbfa1a73-9448-4c8b-9a88-5b5d45d4905f/online


In [52]:
response = client.deployments.score(ai_function_scoring_endpoint, sample_data)

In [53]:
print(response)

{'values': [[[0.014653529971837997], [0], [0.014653529971837997]], [[0.9985828399658203], [1], [0.9985828399658203]]], 'fields': ['prediction', 'prediction_classes', 'probability']}


---
